# Data Cleaning & Feature Engineering
Data cleaning is not the sexiest part of Machine Learning, but it's the part that takes the longest time and the most iterations. It's also one of three most important things that affect your results:
1. Data
2. Features
3. Model

**Better data beats fancier algorithms.** Therefore it is *critial* to take a substantial amount of time in this part of Machine Learning, as it also allows the data scientist to really understand and make discoveries about the data.

### Download the data

The data can be downloaded from UCI's Machine Learning Repository: http://archive.ics.uci.edu/ml/machine-learning-databases/00352/

### Import libraries

In [1]:
import pandas as pd
from datetime import datetime

### Import data

In [3]:
# I like to track how long time codes take to run - especially when running models

start = datetime.now()

df_original = pd.read_excel('../3.customer_segmentation/data/Online Retail.xlsx')

end = datetime.now()
print(end - start)

0:00:47.144645


In [4]:
# Make a copy of the dataframe so we have an original version to compare with, as we make changes to the dataframe

df = df_original

In [5]:
df.sample(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
243153,C558422,23300,GARDENERS KNEELING PAD CUP OF TEA,-1,2011-06-29 11:54:00,1.65,16779.0,United Kingdom
488635,577844,22383,LUNCH BAG SUKI DESIGN,4,2011-11-22 10:41:00,4.96,NaN,United Kingdom


### Clean data
Go through each column and clean/explore the data. 

In [6]:
list(df.columns)

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country']

### Clean column (feature): Quantity
 <s>'InvoiceNo',</s>
 <s>'StockCode',</s>
 <s>'Description',</s>
 **'Quantity',**
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country'

In [7]:
df['Quantity'].describe()

count    541909.000000
mean          9.552250
std         218.081158
min      -80995.000000
25%           1.000000
50%           3.000000
75%          10.000000
max       80995.000000
Name: Quantity, dtype: float64

In [8]:
df[df['Quantity'] < 0].sample()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
352787,C567690,20728,LUNCH BAG CARS BLUE,-6,2011-09-21 17:01:00,1.65,15810.0,United Kingdom


Only keep columns with positive values

In [9]:
# It's always good to check the size before dropping data

df.shape

(541909, 8)

In [10]:
df = df[(df['Quantity']>0)]
df.shape

(531285, 8)

### Clean column: InvoiceDate
 <s>'InvoiceNo',</s>
 <s>'StockCode',</s>
 <s>'Description',</s>
 <s>'Quantity',</s>
 **'InvoiceDate',**
 'UnitPrice',
 'CustomerID',
 'Country'

In [11]:
# Check datatype to see if we need to convert it (e.g. from string to datetime)

df['InvoiceDate'].dtype

dtype('<M8[ns]')

Great! `InvoiceDate` is a datetime data type. No need to convert.

In [12]:
df['InvoiceDate'].describe()

count                  531285
unique                  19052
top       2011-10-31 14:41:00
freq                     1114
first     2010-12-01 08:26:00
last      2011-12-09 12:50:00
Name: InvoiceDate, dtype: object

There's a little over one year's worth of data in the downloaded dataset. To keep things tidy let's restrict the data range to one full year.

One way we can do this is to identify the cut-off date, Loop through each row and if the date is after our cutoff date, we can assign *drop == yes* to it and then drop it. The process would look like this:

1. Create a for-loop that goes through each row and compares InvoiceDate with cutoff_date
2. If date is greater than cutoff_date, assign `df['drop'] == "yes"` to that row
3. Drop rows where *drop == yes*

In [13]:
from datetime import timedelta

In [14]:
# Find out what the cutoff date is (1 year after the earliest date)

df['InvoiceDate'].min() + timedelta(days=365)

Timestamp('2011-12-01 08:26:00')

In [15]:
cutoff_date = df['InvoiceDate'].min() + timedelta(days=365)

In [16]:
# Create a new column that we will use to identify whether or not we should drop the row

df['drop'] = "tbd"

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
df['drop'] = "tbd"

In [74]:
# This would work, but computationally expensive. Try running it and check `docker stats`

for i in df['InvoiceDate']:
    if i > cutoff_date:
        df['drop'] == "yes"
    else:
        df['drop'] == "no"

KeyboardInterrupt: 

As a data scientist, it's important to understand your computer's processing power and know how to employ efficient techniques. The method above maxes out our CPU to 100% and it takes forever to run, especially with larger datasets. CPU is expensive and you don't want to have to pay for more powerful machines when you can just adjust your code.

In [19]:
df = df.drop('drop', axis=1)

In [21]:
# Try a more efficient way and see how much faster it cmomputes

df[df['InvoiceDate'] > cutoff_date].tail(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [22]:
df.shape

(531285, 8)

In [23]:
(df[df['InvoiceDate'] > cutoff_date]).shape

(25135, 8)

In [24]:
# If we drop these rows correctly, we should end up with 506150 rows of data

531285 - 25135

506150

In [27]:
df = df[~(df['InvoiceDate'] > cutoff_date)]

In [25]:
# Alternatively, you can do
# df = df[(df['InvoiceDate'] < cutoff_date)]

In [28]:
df.shape

(506150, 8)

### Clean column: UnitPrice
 <s>'InvoiceNo',</s>
 <s>'StockCode',</s>
 <s>'Description',</s>
 <s>'Quantity',</s>
 <s>'InvoiceDate',</s>
 **'UnitPrice',**
 'CustomerID',
 'Country'

In [29]:
df['UnitPrice'].describe()

count    506150.000000
mean          3.848195
std          42.150519
min      -11062.060000
25%           1.250000
50%           2.080000
75%           4.130000
max       13541.330000
Name: UnitPrice, dtype: float64

In [30]:
df[df['UnitPrice'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [31]:
df[df['UnitPrice'] == 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
1988,536550,85044,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
2024,536552,20950,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
2025,536553,37461,NaN,3,2010-12-01 14:35:00,0.0,NaN,United Kingdom
2026,536554,84670,NaN,23,2010-12-01 14:35:00,0.0,NaN,United Kingdom
4348,536765,84952C,NaN,19,2010-12-02 14:43:00,0.0,NaN,United Kingdom


In [32]:
# Only keep columns with positive values

df.shape

(506150, 8)

In [33]:
df = df[(df['UnitPrice']>0)]
df.shape

(504993, 8)

### Clean column: CustomerID
 <s>'InvoiceNo',</s>
 <s>'StockCode',</s>
 <s>'Description',</s>
 <s>'Quantity',</s>
 <s>'InvoiceDate',</s>
 <s>'UnitPrice',</s>
 **'CustomerID',**
 'Country'

In [34]:
# Check the number of null rows per column/feature

df.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     124413
Country             0
dtype: int64

Looks like some invoices are missing CustomerID. There are many ways of handling null values, such as filling it with the mean/median/mode. However, since our analysis is based on customers, we will have to drop rows with missing CustomerID.

In [35]:
# If we were filling null values with measures of central tendencies, we can use .describe to get summary statistics

df.describe()

,Quantity,UnitPrice,CustomerID
count,504993.000000,504993.000000,380580.000000
mean,10.443963,3.900822,15293.531778
std,111.109818,35.996179,1712.593737
min,1.000000,0.001000,12346.000000
25%,1.000000,1.250000,13969.000000
50%,3.000000,2.080000,15159.000000
75%,11.000000,4.130000,16793.000000
max,74215.000000,13541.330000,18287.000000


In [36]:
# Drop columns that contain null values for CustomerID

df = df.dropna(subset=['CustomerID'], axis=0)

In [37]:
df.shape

(380580, 8)

### Clean column: Country
 <s>'InvoiceNo',</s>
 <s>'StockCode',</s>
 <s>'Description',</s>
 <s>'Quantity',</s>
 <s>'InvoiceDate',</s>
 <s>'UnitPrice',</s>
 <s>'CustomerID',</s>
 **'Country'**

Let's see what countries place the most orders...this will require playing around with the dataframe by slicing (aka indexing), grouping (`df.groupby`) and sorting (`df.sort_values`).

In [38]:
# See how many unique values are in `Country`

df['Country'].nunique()

37

In [39]:
# Why are we missing one Country?

df_original['Country'].nunique()

38

In [40]:
# Find out which Country is missing

set(df_original['Country']) - set(df['Country'])

{'Hong Kong'}

In [41]:
# Explore that Country's data

df_original[df_original['Country']=='Hong Kong']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
69623,541975,22620,4 TRADITIONAL SPINNING TOPS,60,2011-01-24 14:24:00,1.25,NaN,Hong Kong
69624,541975,85183B,CHARLIE & LOLA WASTEPAPER BIN FLORA,6,2011-01-24 14:24:00,3.75,NaN,Hong Kong
69625,541975,85183A,CHARLIE & LOLA WASTEPAPER BIN BLUE,6,2011-01-24 14:24:00,3.75,NaN,Hong Kong
69626,541975,16236,KITTY PENCIL ERASERS,28,2011-01-24 14:24:00,0.21,NaN,Hong Kong
69627,541975,21058,PARTY INVITES WOODLAND,12,2011-01-24 14:24:00,0.85,NaN,Hong Kong
69628,541975,21967,PACK OF 12 SKULL TISSUES,24,2011-01-24 14:24:00,0.29,NaN,Hong Kong
69629,541975,21981,PACK OF 12 WOODLAND TISSUES,24,2011-01-24 14:24:00,0.29,NaN,Hong Kong
69630,541975,22553,PLASTERS IN TIN SKULLS,36,2011-01-24 14:24:00,1.65,NaN,Hong Kong
69631,541975,22556,PLASTERS IN TIN CIRCUS PARADE,36,2011-01-24 14:24:00,1.65,NaN,Hong Kong
69632,541975,22554,PLASTERS IN TIN WOODLAND ANIMALS,36,2011-01-24 14:24:00,1.65,NaN,Hong Kong


In [42]:
# Looks like we dropped all of Hong Kong's data because they never inputted the CustomerID field

df_original[df_original['Country']=='Hong Kong']['CustomerID'].unique()

array([ nan])

### Re-index dataframe
After dropping all columns, it's important to re-index your dataframe.

In [43]:
df = df.reset_index()

### Feature engineering
When you use domain knowledge of the data to create features that help the algorithm work better. The quality and quantity of features will have great influence on whether the model is good or not. Choosing the right features can produce simpler and more flexible models, and often yield better results.

Three components that affect results are:
1. Data
2. Features
3. Model

In [46]:
df['InvoiceDate'].isnull().unique()

array([False], dtype=object)

In [47]:
df.shape

(380580, 9)

In [48]:
df['InvoiceDate'].shape

(380580,)

In [49]:
month_list = []
dayofweek_list = []
hour_list = []

for i in df['InvoiceDate']:
    month_list.append(datetime.date(i).month)
    dayofweek_list.append(datetime.weekday(i))
    hour_list.append(datetime.time(i).hour)

In [50]:
len(month_list)

380580

In [51]:
month_list[380570:380580]

[11, 11, 11, 11, 11, 11, 11, 11, 11, 11]

In [52]:
pd.Series(month_list)[380570:380580]

380570    11
380571    11
380572    11
380573    11
380574    11
380575    11
380576    11
380577    11
380578    11
380579    11
dtype: int64

In [53]:
df['Month'] = pd.Series(month_list)
df['Dayofweek'] = pd.Series(dayofweek_list)
df['Hour'] = pd.Series(hour_list)

In [54]:
pd.Series(month_list)

0         12
1         12
2         12
3         12
4         12
5         12
6         12
7         12
8         12
9         12
10        12
11        12
12        12
13        12
14        12
15        12
16        12
17        12
18        12
19        12
20        12
21        12
22        12
23        12
24        12
25        12
26        12
27        12
28        12
29        12
          ..
380550    11
380551    11
380552    11
380553    11
380554    11
380555    11
380556    11
380557    11
380558    11
380559    11
380560    11
380561    11
380562    11
380563    11
380564    11
380565    11
380566    11
380567    11
380568    11
380569    11
380570    11
380571    11
380572    11
380573    11
380574    11
380575    11
380576    11
380577    11
380578    11
380579    11
dtype: int64

In [55]:
df.tail()

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Dayofweek,Hour
380575,516364,579885,85034C,3 ROSE MORRIS BOXED CANDLES,4,2011-11-30 17:37:00,1.25,15444.0,United Kingdom,11,2,17
380576,516365,579885,21742,LARGE ROUND WICKER PLATTER,2,2011-11-30 17:37:00,5.95,15444.0,United Kingdom,11,2,17
380577,516366,579885,23084,RABBIT NIGHT LIGHT,6,2011-11-30 17:37:00,2.08,15444.0,United Kingdom,11,2,17
380578,516367,579885,21257,VICTORIAN SEWING BOX MEDIUM,1,2011-11-30 17:37:00,7.95,15444.0,United Kingdom,11,2,17
380579,516368,579885,21259,VICTORIAN SEWING BOX SMALL,1,2011-11-30 17:37:00,5.95,15444.0,United Kingdom,11,2,17


### Export the cleaned data

In [56]:
from sklearn.externals import joblib

In [58]:
joblib.dump(df, '../3.customer_segmentation/data/df.p')

['../3.customer_segmentation/data/df.p']

In [59]:
joblib.dump(df_original, '../3.customer_segmentation/data/df_original.p')

['../3.customer_segmentation/data/df_original.p']